In [1]:
import pandas as pd

In [113]:
df = pd.read_csv('crime_data.csv')
df.drop(columns=["Unnamed: 0"], inplace=True)
df = df.replace(to_replace=r"AM", value="", regex=True)
df = df.replace(to_replace=r"PM", value="", regex=True)

In [120]:
df = df[df["Time_of_Incident"].str.contains("Unknown") == False]

In [121]:
df["Time_of_Incident"][50:200]

50     2/28/2016 16:00
52       3/1/2016 0:00
53       3/2/2016 1:00
54       3/4/2016 0:30
55       3/4/2016 2:15
            ...       
203     9/19/2016 2:20
204     9/20/2016 0:00
205    9/20/2016 22:30
206    9/21/2016 16:30
207    9/21/2016 23:00
Name: Time_of_Incident, Length: 150, dtype: object

In [3]:
df

,Unnamed: 0,Time_of_Incident,Time_of_report,Incident,Location
0,0,1/8/2016 7:00,12/20/2016,Sex Offense,7100 Blk Preinkert Drive
1,1,1/21/2016 4:00,3/3/2017,Vandalism,South Campus Commons
2,2,1/22/2016 18:00,3/3/2017,Alcohol Violation,Wicomico Hall
3,3,1/22/2016 20:54,3/3/2017,Alcohol Violation,Easton Hall
4,4,1/22/2016 21:00,3/3/2017,Alcohol Violation,Ellicott Hall
...,...,...,...,...,...
1967,1967,9/29/2023 18:00,10/12/2023,Theft From Auto,4200 Blk Knox Road
1968,1968,10/1/2023 12:00,10/12/2023,Theft From Auto,4300 Blk Calvert Road
1969,1969,10/6/2023 5:37,10/12/2023,Theft From Auto,9000 Blk Baltimore Avenue
1970,1970,10/6/2023 23:00,10/12/2023,Theft From Auto,4300 Blk Hartwick Road
